<h2 align="center">Ejercicio Investigadores de la UIB</h2> <h3 align="center"> Arturo González Moya</h3> 

El ejercicio consiste en acceder a las páginas de la UIB en donde se detallan los datos de los investigadores. De cada uno de ellos se deberá recoger información como el nombre, el género (si es hombre o mujer), el título (Dra., Sr., ...), a qué grupo de investigación pertenece, su currículum vitae breve, que rol tiene en el grupo de investigación, ...

Lo primero que hacemos es incorporar las herramientas que necesitaremos para realizar esta tarea.

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random

Añadiremos también una función para grabar la información a un fichero y otra para cargar un fichero.

In [3]:
def dump_data(data, filename):
    with open(filename, "w") as out_file:
        json.dump(data, out_file)

def load_data(filename):
    data = None
    with open(filename, "r") as in_file:
        data = json.load(in_file)
    return data

Comenzamos con lo que sería el trabajo. La información de los investigadores la podemos encontrar en el siguiente enlace:

url_categorias = "https://www.uib.cat/recerca/estructures/grups/grups_area/id_area=-1%2526npag={}"

desde el que se puede acceder a todos los grupos de investigación. La organización de estos grupos viene en diferentes páginas ya que hay gran cantidad de ellos. Además, la página de la UIB puede estar en castellano, catalán o inglés. Los CV de los investigadores los extraeremos en los 3 idiomas.

In [4]:
url_categorias = "https://www.uib.cat/recerca/estructures/grups/grups_area/id_area=-1%2526npag={}"

Usaremos la siguiente función para cargar las páginas web.

In [5]:
def load_page(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'} #Añadimos esto por problemas al cargar las páginas
    pagina_categorias = requests.get(url,headers = headers)
    if pagina_categorias.status_code == 200:
        return BeautifulSoup(pagina_categorias.text,'html5')
    else:
        print("Esta página no existe. Cambiamos a una de la lista")
        return None

Cargamos la página anterior.

In [6]:
categorias = load_page(url_categorias)

In [7]:
categorias

<!DOCTYPE html>
<html dir="ltr" itemscope="" itemtype="http://schema.org/WebPage" lang="ca"><head>
  <title itemprop="name">Llista de grups de recerca segons àrea AEI - Grups de recerca - Estructures - R+D+I -  Universitat de les Illes Balears</title>
  <meta charset="utf-8"/>
  <meta content="Universitat de les Illes Balears" name="author"/>
  <meta content="Llista de grups de recerca segons àrea AEI - Grups de recerca - Estructures - R+D+I -  Universitat de les Illes Balears" name="DCTERMS.title"/>
  <meta content="Universitat de les Illes Balears" name="DCTERMS.creator"/>
  <meta content="ca" name="DC.language"/>
  <meta content="all, index, follow" name="robots"/><meta content="always" name="referrer"/>
  <meta content="39.636344;2.646333" name="ICBM"/>
  <meta content="39.636344;2.646333" name="geo.position"/>
  <meta content="ES-PM" name="geo.region"/>
  <meta content="Universitat de les Illes Balears" name="geo.placename"/>

  <meta content="Llista de grups de recerca segons àre

Podemos ver que las categorias se encuentran en un elemento *div* con clase *uib_style_filaunica*

In [8]:
contenedor = categorias.find("div", class_="uib_style_filaunica")

En este elemento, se organizan como *li*

In [9]:
grupos = contenedor.find_all("li")

In [10]:
grupos

[<li>
 <a href="https://www.uib.cat/recerca/estructures/grups/grup/GENEHUMA/">Genètica humana</a>
 </li>,
 <li>
 <a href="https://www.uib.cat/recerca/estructures/grups/grup/Microbio/">Microbiologia (Microbio)</a>
 </li>,
 <li>
 <a href="https://www.uib.cat/recerca/estructures/grups/grup/BIOCANCER/">Biologia Cel·lular del Càncer (BIOCANCER)</a>
 </li>,
 <li>
 <a href="https://www.uib.cat/recerca/estructures/grups/grup/MOLONE/">Biologia molecular i salut global</a>
 </li>,
 <li>
 <a href="https://www.uib.cat/recerca/estructures/grups/grup/CurES/">Cures, cronicitat i evidències en salut (CurES)</a>
 </li>,
 <li>
 <a href="https://www.uib.cat/recerca/estructures/grups/grup/EVES/">Evidència, Estils de Vida i Salut</a>
 </li>,
 <li>
 <a href="https://www.uib.cat/recerca/estructures/grups/grup/GMOT/">Grup Multidisciplinar d'Oncologia Translacional (GMOT)</a>
 </li>,
 <li>
 <a href="https://www.uib.cat/recerca/estructures/grups/grup/INFEINMU/">Infecció i immunitat</a>
 </li>,
 <li>
 <a href="h

Cada unos de los grupos tiene un enlace. Por ejemplo el primer enlace que podemos ver es el siguiente

In [11]:
grupos[0].find("a").get("href")

'https://www.uib.cat/recerca/estructures/grups/grup/GENEHUMA/'

Añadimos la función creada en clase para extraer el grupo

In [12]:
def extrae_grupo(item):
    nombre = item.text.strip()
    url = item.find("a").get("href")
    return nombre, url

También añadimos una función que nos indica si hay una siguiente página con más grupos.

In [13]:
def hay_siguiente(pagina):
    return pagina.find("img", title="Pàg. Següent") is not None

Una vez tenemos la url donde se encuentra la información del grupo de investigación, tenemos que encontrar donde está el equipo de investigación para obtener lo que se nos pide. Añadiendo "/equip/index.html" a la url obtenida anteriormente podemos entrar a la información del grupo de investigación, pero esto no ocurre con todas las páginas ya que tenemos varias excepciones.

Estas excepciones son las siguientes:

* "https://engalim.uib.cat/Grupo-investigador/"

* "https://gicafe.uib.cat/Estructura_I_Personal/"

* "https://eic.uib.cat/Equip-investigador/"

* "https://gedhe.uib.cat/Equip-investigador/"

* "https://gifes.uib.cat/Research-staff/"

* "https://meteo.uib.cat/equip/"

* "https://recumare.uib.cat/EquipInvestigador/"

* "https://reacmole.uib.cat/Equip-investigador/"

* "https://ccts.uib.cat/Equipo-investigador/"

* "https://desigualtats.uib.cat/Equip-investigador/"

* "https://gresib.uib.cat/Equip-investigador/"

* "https://relatmit.uib.cat/Equip-investigador/"

* "https://grupestudidha.uib.cat/Equip-investigador/"

* "https://praxis.uib.cat/Equip/"

* "https://imasdel.uib.cat/Equip/"

* "https://ndpc.uib.cat/Equip-investigador/"

* "https://acsic.uib.cat/EquipInv/"

* "https://scopia.uib.cat/Equip/"

Guardaremos en una lista todas estas excepciones.

In [14]:
lista_excepciones = ["https://engalim.uib.cat/Grupo-investigador/","https://gicafe.uib.cat/Estructura_I_Personal/",
                       "https://eic.uib.cat/Equip-investigador/", "https://gedhe.uib.cat/Equip-investigador/",
                       "https://gifes.uib.cat/Research-staff/","https://meteo.uib.cat/equip/",
                       "https://recumare.uib.cat/EquipInvestigador/", "https://reacmole.uib.cat/Equip-investigador/",
                       "https://ccts.uib.cat/Equipo-investigador/", "https://desigualtats.uib.cat/Equip-investigador/",
                       "https://gresib.uib.cat/Equip-investigador/", "https://relatmit.uib.cat/Equip-investigador/",
                       "https://grupestudidha.uib.cat/Equip-investigador/", "https://praxis.uib.cat/Equip/",
                       "https://imasdel.uib.cat/Equip/", "https://ndpc.uib.cat/Equip-investigador/",
                       "https://acsic.uib.cat/EquipInv/", "https://scopia.uib.cat/Equip/"]

Accedemos a la página de los investigadores de la que hemos tomado de ejemplo.

In [15]:
pgrupo_url = grupos[0].find("a").get("href") + "/equip/index.html"

In [16]:
pgrupo = load_page(pgrupo_url)

La información que nos interesa se encuentra dentro de un elemento *div* con atributo *itemprop* y con valor *mainContentOfPage*

In [17]:
contenido = pgrupo.find("div", itemprop="mainContentOfPage")

In [18]:
lista_rel = contenido.find_all("h3")

In [19]:
lista_rel

[<h3>Investigadora principal</h3>, <h3>Membres</h3>, <h3>Col·laboradors</h3>]

Las personas que se encuentran en cada rango dentro del grupo de investigación son las siguientes a cada elemento de la lista anterior.

In [20]:
personal = lista_rel[2].find_next_sibling().find_all("li")

In [21]:
personal

[<li><a class="link_fitxa" href="https://www.uib.cat/personal/ABjIwNDQxMw/">Dr. José Antonio Jurado Rivera</a> (Professor contractat doctor interí)</li>,
 <li><a class="link_fitxa" href="https://www.uib.cat/personal/ABjIwNDE4NA/">Dra. Joana Francesca Ferragut Simonet</a> (Professora ajudanta doctora)</li>,
 <li>Dra. Maria Magdalena Parera Rossello (Professora associada)</li>,
 <li><a class="link_fitxa" href="https://www.uib.cat/personal/ABjIyMTkwNA/">Sra. Iris Alemany Hermoso</a> (CAIB -&gt; Contractada predoctoral)</li>,
 <li><a class="link_fitxa" href="https://www.uib.cat/personal/ABjIwNDA3MQ/">Sra. Joana Maria Buades Payeras</a> (CAIB -&gt; Contractada predoctoral)</li>,
 <li><a class="link_fitxa" href="https://www.uib.cat/personal/ABTIxNzM3/">Sra. Virginia Rodríguez Delgado</a> (CAIB -&gt; Contractada predoctoral)</li>,
 <li>Sra. Beatriz Sánchez Barceló (Professora associada)</li>]

Ahora realizaremos un estudio de cada individuo en el grupo de investigación que hemos elegido de prueba. Lo primero será sacar el nombre del investigador. Lo haremos de ejemplo para la tercera persona.

In [22]:
persona = personal[2].text

In [23]:
partes = persona.split('(')

In [24]:
p1 = partes[0].strip().split()

In [25]:
p1

['Dra.', 'Maria', 'Magdalena', 'Parera', 'Rossello']

Agrupamos el nombre

In [26]:
nombre = " ".join(p1[1:])

In [27]:
nombre

'Maria Magdalena Parera Rossello'

El primer elemento de la lista nos puede servir para determinar el género del individuo. Utilizaremos la función siguiente que vimos en clase.

In [28]:
def genero(p):
    if p[2]=='a':
        return "FEMENINO"
    else:
        return "MASCULINO"

In [29]:
genero(p1[0])

'FEMENINO'

La otra sección que tenemos en la variable **partes** nos indica el vínculo que el individuo tiene con la universidad.

In [30]:
partes[1][:-1]

'Professora associada'

Por último, hemos de obtener el Curriculum Vitae de cada investigador. Ya que puede ser que haya CV que estén en un idioma y no en otro, lo que haremos será leer los CV de los 3 idiomas. En el ejemplo leeremos solo el CV en catalán.

Lo primero que hemos de hacer es tomar la página donde se encuentra la información del investigador. En nuestro ejemplo sería la siguiente

In [31]:
url = personal[0].find("a").get('href')

Cargamos la página y utilizamos la siguiente función vista en clase (con unas modificaciones) para obtener el CV. En el caso de que no exista CV, se puede encontrar un elemento *div* de clase *uib_style_nolanguageversion*.

In [32]:
pagina_cv = load_page(url)

In [33]:
def extrae_cv(url):
    pagina_cv = load_page(url)
    if pagina_cv.find("div", class_ = "uib_style_nolanguageversion") is not None:
        return "No existe CV en este idioma"
    else:
        if pagina_cv.find("div", id = "cv_breve") is not None:
            return pagina_cv.find("div", id = "cv_breve").text
        else:
            return "No tiene CV en este idioma"

En el ejemplo que estamos tratando, obtenemos el siguiente CV

In [34]:
extrae_cv(url)

"Llicenciat en Biologia per la Universidad de Granada (2002) i Doctor en Biologia per la UIB (2015). Ha realitzat estades de recerca a British Natural History Museum (London, UK), Australian Museum (Sydney, Australia), Western Australian Museum (Perth, Australia), Reserva Biológica Alberto Manuel Brenes (Costa Rica), Institut de Biologia Evolutiva (CSIC, Barcelona), Institut Agronomique Neo-Caledonien (Nova Caledònia) finançat pel Govern Espanyol, per la UE, i per la National Geographic Society. Des de 2009 a 2014 va estar vinculat amb contractes de recerca a l'Institut Mediterrani d'Estudis Avançats (IMEDEA, CSIC). Durant 5 anys va ser Professor Associat del Departament de Biologia (Genètica) de la UIB (2010-2014). Va posteriorment obtenir un contracte com a Professor Ajudant Doctor a la mateixa institució (2015). Durant l'etapa doctoral va treballar en l'estudi molecular basat en seqüències d'ADN de les relacions evolutives i la història biogeogràfica d'artròpodes. A més, va desenvol

Aplicando todos estos pasos a cada grupo de investigación, obtendríamos lo que buscamos.

Utilizaremos la siguiente función para extraer los datos de cada investigador y guardarlos como un diccionario.

In [35]:
def extrae_investigador(item,tipo, idioma, grupo):
    if item.find("a", class_="link_fitxa") is not None:
        url = item.find("a").get('href')
    else:
        url = None
    texto = item.text
    partes = texto.split("(")
    p1= partes[0].strip().split()
    nombre = " ".join(p1[1:])
    gener = genero(p1[0])
    if tipo.text[2]== 'v': # De esta forma agrupamos en una etiqueta a todos los investigadores principales.
        tipo_investigador = "Investigador/a principal"
    else:
        tipo_investigador = tipo.text
    titulo = p1[0]
    if len(partes)>1: # Añadimos este if porque hay individuos en los que no aparece su relación con la UIB.
        relacion_uib = partes[1][:-1]
    else:
        relacion_uib = "No tiene"
    cv=[]
    if url is not None: # Extraer el CV en catalan
        cv.append(extrae_cv(url))
    else:
        cv.append("No tiene CV en este idioma")
    for i in (0,1): # Extraer el CV en Castellano e Ingles.
        if url is not None:
            url = url.replace(idioma[i], idioma[i+1])
            cv.append(extrae_cv(url))
        else:
            cv.append("No tiene CV en este idioma")
    dicc = {"Grupo": grupo, "Nombre": nombre, "Genero": gener,
                    "Tipo de investigador": tipo_investigador,  "Titulo" : titulo,
                    "Relacion UIB": relacion_uib, "CV_cat": cv[0], "CV_esp": cv[1], "CV_eng": cv[2]}
    return dicc

La función está basada en la que se hizo en clase pero con pequeñas modificaciones que se han necesitado por los siguientes motivos:

* Investigadores que no tiene pagína donde encontrar sus datos

* Investigadores en los que no aparece su vínculo con la UIB

Utilizaremos la siguiente función (que ya empleamos para extraer los chistes) para recopilar los datos de las diferentes páginas de grupos de investigación.

In [36]:
def parse_pages():
    pagina = 1
    while parse_page(pagina):
        time.sleep(random.randrange(1, 3))
        pagina = pagina + 1

Definimos la siguiente función para recorrer y recoger los datos de todas la páginas

In [37]:
def parse_page(pagina):
    print("parsing page ", pagina)
    categorias = load_page(url_categorias.format(pagina))
    time.sleep(random.randrange(1, 3))
    contenedor = categorias.find("div", class_="uib_style_filaunica")
    grupos = contenedor.find_all("li")
    for c in grupos:
        a = extrae_grupo(c)
        pgrupo_url = a[1] + "/equip/index.html"
        idiomas = [".cat", ".es/es", ".eu"] # Para leer los cv de todos los idiomas
        pgrupo = load_page(pgrupo_url)
        time.sleep(random.randrange(1, 3))
        if pgrupo is None: # Esta condición sirve para poder leer las paginas que no siguen lo habitual.
            nueva_pagina = lista_excepciones[0]
            idiomas = [".cat", ".es", ".eu"] # Si cambia la pagina hay que cambiar tambien la forma de las extensiones
            pgrupo = load_page(nueva_pagina)
            lista_excepciones.pop(0)
        contenido = pgrupo.find("div", itemprop="mainContentOfPage")
        lista_rel = contenido.find_all("h3")
        for i in lista_rel:
            time.sleep(random.randrange(1, 4))
            personal = i.find_next_sibling().find_all("li")
            for j in personal:
                investigador = extrae_investigador(j,i, idiomas,a[0])
                datos.append(investigador)
    return hay_siguiente(categorias)

Comenzamos a crear el diccionario

In [38]:
datos = []
parse_pages()

parsing page  1
parsing page  2
Esta página no existe. Cambiamos a una de la lista
parsing page  3
Esta página no existe. Cambiamos a una de la lista
Esta página no existe. Cambiamos a una de la lista
Esta página no existe. Cambiamos a una de la lista
Esta página no existe. Cambiamos a una de la lista
parsing page  4
parsing page  5
Esta página no existe. Cambiamos a una de la lista
parsing page  6
Esta página no existe. Cambiamos a una de la lista
Esta página no existe. Cambiamos a una de la lista
parsing page  7
Esta página no existe. Cambiamos a una de la lista
Esta página no existe. Cambiamos a una de la lista
Esta página no existe. Cambiamos a una de la lista
parsing page  8
Esta página no existe. Cambiamos a una de la lista
parsing page  9
parsing page  10
parsing page  11
Esta página no existe. Cambiamos a una de la lista
parsing page  12
Esta página no existe. Cambiamos a una de la lista
parsing page  13
parsing page  14
Esta página no existe. Cambiamos a una de la lista
Esta p

Veamos lo que hemos obtenido

In [39]:
datos

[{'Grupo': 'Genètica humana',
  'Nombre': 'Cori Ramon Juanpere',
  'Genero': 'FEMENINO',
  'Tipo de investigador': 'Investigador/a principal',
  'Titulo': 'Dra.',
  'Relacion UIB': "Catedràtica d'universitat",
  'CV_cat': 'Llicenciada en Ciencies per la UAB i doctora en Ciencies per la UIB. Catedràtica de Genètica a la UIB. Ha impartit docencia de materies de Genètica i Evolució, a llicenciatura Grau i Posgrau/Doctorat. La recerca està centrada en la Genètica poblacional i evolutiva amb especial enfasis a la diferenciació en els entorns insulars.',
  'CV_esp': 'No existe CV en este idioma',
  'CV_eng': 'No existe CV en este idioma'},
 {'Grupo': 'Genètica humana',
  'Nombre': 'José Aurelio Castro Ocón',
  'Genero': 'MASCULINO',
  'Tipo de investigador': 'Membres',
  'Titulo': 'Dr.',
  'Relacion UIB': "Catedràtic d'universitat",
  'CV_cat': "El camp general d'investigació que he desenvolupat en els darrers anys està relacionat amb la Genètica de Poblacions i l'Evolució, tant de poblacion

Para acabar, guardamos los datos en un fichero *.json*.

In [40]:
filename = 'Entrega_UIB_con_varios_cv.json'

In [41]:
dump_data(datos, filename)

Veamos cuantos elementos tiene nuestro conjunto de datos.

In [42]:
len(datos)

2260